## Lab 13c: Model-Free Markowitz using Reinforcement Learning (RL)

## Setup

In [1]:
include("CHEME-5760-L13c-CodeLibrary.jl");

In [2]:
# add -
import Pkg;
required_packages = ["JLD2","FileIO","DataFrames","CSV"];
for package ∈ required_packages
    Pkg.add(package)
end

# import -
using JLD2,FileIO,DataFrames,CSV

# Paths 
const _ROOT = pwd();
const _PATH_TO_SRC = joinpath(_ROOT, "src");
const _PATH_TO_DATA = joinpath(_ROOT, "data");

   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Manifest.toml`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Labs-F23/Manifest.toml`


## Task 1: Load My Sample Portfolio File and the Efficient Frontier

In [3]:
tmp = load(joinpath(_PATH_TO_DATA,"MyChoiceSet-Example.jld2")) |> x-> x["dataset"];
μ̂ = tmp["expected_excess_return"];
Σ̂ = tmp["covariance"];

In [4]:
portfolio_df = load(joinpath(_PATH_TO_DATA,"MyEfficientFrontier-Example.jld2")) |> x-> x["dataset"];

In [5]:
portfolio_df

Row,expected_excess_return,risk,tickers,w,risk_free_rate
,Float64,Float64,Array…,Array…,Float64
1,0.00404809,0.162549,"[""PFE"", ""MRK"", ""AMD""]","[0.421066, 0.564606, 0.0143277]",0.05
2,0.0115386,0.16276,"[""PFE"", ""MRK"", ""AMD""]","[0.397039, 0.572796, 0.0301658]",0.05
3,0.023077,0.163904,"[""PFE"", ""MRK"", ""AMD""]","[0.360028, 0.58541, 0.0545629]",0.05
4,0.0346154,0.166024,"[""PFE"", ""MRK"", ""AMD""]","[0.323016, 0.598024, 0.0789601]",0.05
5,0.0461539,0.169082,"[""PFE"", ""MRK"", ""AMD""]","[0.286005, 0.610637, 0.103358]",0.05
6,0.0576923,0.173029,"[""PFE"", ""MRK"", ""AMD""]","[0.248994, 0.623251, 0.127755]",0.05
7,0.0692308,0.177804,"[""PFE"", ""MRK"", ""AMD""]","[0.211983, 0.635865, 0.152152]",0.05
8,0.0807692,0.183345,"[""PFE"", ""MRK"", ""AMD""]","[0.174971, 0.648479, 0.176549]",0.05
9,0.0923077,0.189583,"[""PFE"", ""MRK"", ""AMD""]","[0.13796, 0.661093, 0.200947]",0.05


In [6]:
number_of_divisions = 21
number_of_stocks = 3
total_budget = 1.0;
softwall_reward = -2000.0;
defaultreward = 0.0;
dimensions = Dict{Int,Int}();
dimensions[1] = number_of_divisions;
dimensions[2] = number_of_divisions;
dimensions[3] = number_of_divisions;

nactions = 7; 
nstates = number_of_divisions^number_of_stocks;
𝒮 = range(1,stop=nstates,step=1) |> collect;
𝒜 = range(1,stop=nactions,step=1) |> collect;
w = range(0.0,stop=1.0,length=number_of_divisions) |> collect;
rewards = Dict{Tuple{Int,Int,Int},Float64}();

In [7]:
world_model = build(MyCubicGridWorldModel, (
        dimensions = dimensions, rewards = rewards, defaultreward = defaultreward
));

In [8]:
world_model.states

Dict{Tuple{Int64, Int64, Int64}, Int64} with 9261 entries:
  (19, 5, 6)   => 8028
  (6, 13, 14)  => 2471
  (3, 9, 8)    => 1058
  (20, 16, 1)  => 8695
  (16, 15, 16) => 6925
  (14, 12, 1)  => 5965
  (11, 13, 15) => 4677
  (16, 5, 3)   => 6702
  (18, 6, 19)  => 7621
  (20, 6, 5)   => 8489
  (9, 4, 21)   => 3612
  (7, 10, 9)   => 2844
  (10, 1, 21)  => 3990
  (17, 9, 9)   => 7233
  (7, 13, 12)  => 2910
  (18, 6, 10)  => 7612
  (3, 14, 2)   => 1157
  (2, 18, 9)   => 807
  (10, 6, 8)   => 4082
  (3, 9, 21)   => 1071
  (5, 14, 11)  => 2048
  (11, 19, 13) => 4801
  (6, 12, 17)  => 2453
  (10, 3, 12)  => 4023
  (20, 12, 4)  => 8614
  ⋮            => ⋮

## Task 2: Build the Q-learning agent

In [9]:
α = 0.70;  # learning rate
γ = 0.95; # discount rate
agent_model = build(MyQLearningAgentModel, (
    states = 𝒮,
    actions = 𝒜,
    α = α,
    γ = γ,
    Q = zeros(nstates,nactions) # Hmmm. Could we do something better that this?
));

## Task 3: Update the world model

In [10]:
# setup soft walls (budget constraints)
soft_wall_set = Set{Tuple{Int, Int, Int}}();
for s ∈ 𝒮
    
    # get the position -
    current_position = world_model.coordinates[s];
    
    # compute the w_array -
    w_array = Array{Float64,1}()
    for i in current_position
        push!(w_array, w[i]);
    end
    
    # does this position violate the budget?
    budget_violation = max(0.0, sum(w_array) - total_budget)
    if (budget_violation ≥ 1.0)
        push!(soft_wall_set, current_position)
    end
end

for s ∈ 𝒮
    current_position = world_model.coordinates[s]
    for a ∈ 𝒜
        Δ = world_model.moves[a];
        new_position =  current_position .+ Δ
    
        if (in(new_position, soft_wall_set) == true)
          rewards[new_position] = softwall_reward  
        end
    end
end


In [11]:
world_model.coordinates

Dict{Int64, Tuple{Int64, Int64, Int64}} with 9261 entries:
  4986 => (12, 7, 9)
  7329 => (17, 13, 21)
  4700 => (11, 14, 17)
  4576 => (11, 8, 19)
  7144 => (17, 5, 4)
  6073 => (14, 17, 4)
  2288 => (6, 4, 20)
  1703 => (4, 19, 2)
  1956 => (5, 10, 3)
  8437 => (20, 3, 16)
  2350 => (6, 7, 19)
  7685 => (18, 9, 20)
  8690 => (20, 15, 17)
  5975 => (14, 12, 11)
  3406 => (8, 16, 4)
  2841 => (7, 10, 6)
  2876 => (7, 11, 20)
  687  => (2, 12, 15)
  7353 => (17, 15, 3)
  185  => (1, 9, 17)
  1090 => (3, 10, 19)
  2015 => (5, 12, 20)
  7272 => (17, 11, 6)
  8544 => (20, 8, 18)
  7958 => (19, 1, 20)
  ⋮    => ⋮

In [12]:
# do some shaping?
is_reward_shaping_on = true;
if (is_reward_shaping_on == true)
    for x in 1:number_of_divisions
        for y in 1:number_of_divisions
            for z in 1:number_of_divisions
                
               coordinate = (x,y,z);
                if (haskey(rewards, coordinate) == false && in(coordinate, soft_wall_set) == false)

                    # compute the w_array -
                    w_array = Array{Float64,1}()
                    for i in coordinate
                        push!(w_array, w[i]);
                    end

                    # compute the risk, and return values -
                    risk_value = transpose(w_array)*Σ̂*w_array;
                    return_value = transpose(w_array)*μ̂
                    sharpe_ratio = return_value/risk_value

                    # compute the reward -
                    rewards[coordinate] = defaultreward + sharpe_ratio
                end 
            end
        end
    end
end

In [13]:
rewards;

## Task 4: Simulate

In [14]:
startstate = (1,1,1); # start position
number_of_episodes = 100;
number_of_iterations = 1000;

In [15]:
my_Q_dictionary = Dict{Tuple{Int,Int,Int}, Array{Float64,2}}();
coordinate = startstate;
for i ∈ 1:number_of_episodes
    
    # run an episode, and grab the Q
    result = simulate(agent_model, world_model, coordinate, number_of_iterations, ϵ = 0.9);

    # update the agent with the Q from the last episode, and try to refine this Q
    # Analogy: practice make perfect ...
    agent_model.Q = result.Q;
end
my_Q_dictionary[coordinate] = agent_model.Q;

In [16]:
my_Q_dictionary

Dict{Tuple{Int64, Int64, Int64}, Matrix{Float64}} with 1 entry:
  (1, 1, 1) => [0.0 -1.0e12 … -1.0e12 0.0; 0.0 -1.0e12 … 0.0 0.0; … ; -9.73e11 …

In [17]:
Q = my_Q_dictionary[startstate];
my_π = policy(Q);

In [22]:
action_label_dict = Dict{Int,String}()
action_label_dict[1] = "↑ A1"
action_label_dict[2] = "↓ A1"
action_label_dict[3] = "↑ A2"
action_label_dict[4] = "↓ A2"
action_label_dict[5] = "↑ A3"
action_label_dict[6] = "↓ A3"
action_label_dict[7] = "∅";

In [25]:
coordinates = world_model.coordinates;
moves = world_model.moves;
for s ∈ 𝒮
    action = my_π[s];   
    println("action = $(action)")
end

action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1
action = 1